## Fitting a minimal radiative model

In [1]:
#!/usr/bin/env python
import astropy.units as u
import numpy as np
from astropy.io import ascii

import naima
from naima.models import ExponentialCutoffPowerLaw, InverseCompton

## Read data

data = ascii.read("RXJ1713_HESS_2007.dat")


def ElectronIC(pars, data):
    """
    Define particle distribution model, radiative model, and return model flux
    at data energy values
    """

    ECPL = ExponentialCutoffPowerLaw(
        pars[0] / u.eV, 10.0 * u.TeV, pars[1], 10 ** pars[2] * u.TeV
    )
    IC = InverseCompton(ECPL, seed_photon_fields=["CMB"])

    return IC.flux(data, distance=1.0 * u.kpc)


def lnprior(pars):
    # Limit amplitude to positive domain
    logprob = naima.uniform_prior(pars[0], 0.0, np.inf)
    return logprob


In [2]:
if __name__ == "__main__":

    ## Set initial parameters and labels
    p0 = np.array((1e30, 3.0, np.log10(30)))
    labels = ["norm", "index", "log10(cutoff)"]

    ## Run sampler
    sampler, pos = naima.run_sampler(
        data_table=data,
        p0=p0,
        labels=labels,
        model=ElectronIC,
        prior=lnprior,
        nwalkers=32,
        nburn=100,
        nrun=20,
        threads=4,
        prefit=True,
        interactive=False,
    )
    ## Save run results
    out_root = "RXJ1713_IC_minimal"
    naima.save_run(out_root + ".hdf5", sampler)

    ## Save diagnostic plots and results table
    naima.save_diagnostic_plots(out_root, sampler, sed=False)
    naima.save_results_table(out_root, sampler)

INFO: Finding Maximum Likelihood parameters through Nelder-Mead fitting... [naima.core]
INFO:    Initial parameters: [4.74159485e+32 3.00000000e+00 1.47712125e+00] [naima.core]
INFO:    Initial lnprob(p0): -177.635 [naima.core]
INFO:    New ML parameters : [2.60447920e+32 2.61876368e+00 1.71023212e+00] [naima.core]
INFO:    Maximum lnprob(p0): -17.902 [naima.core]
Burning in the 32 walkers with 100 steps...

Progress of the run: 0 percent (0 of 100 steps)
                           -----norm------ -----index----- -log10(cutoff)-
  Last ensemble median :    2.61e+32          2.62            1.71      
  Last ensemble std    :    2.17e+30         0.0115          0.0105     
  Last ensemble lnprob :  avg: -18.399, max: -17.900

Progress of the run: 5 percent (5 of 100 steps)
                           -----norm------ -----index----- -log10(cutoff)-
  Last ensemble median :    2.61e+32          2.62            1.71      
  Last ensemble std    :    6.51e+30         0.0502          0.0232  


Progress of the run: 25 percent (5 of 20 steps)
                           -----norm------ -----index----- -log10(cutoff)-
  Last ensemble median :     2.6e+32          2.66            1.72      
  Last ensemble std    :     1.4e+31          0.114          0.0975     
  Last ensemble lnprob :  avg: -19.674, max: -17.975

Progress of the run: 30 percent (6 of 20 steps)
                           -----norm------ -----index----- -log10(cutoff)-
  Last ensemble median :    2.61e+32          2.67            1.72      
  Last ensemble std    :    1.36e+31          0.113          0.0938     
  Last ensemble lnprob :  avg: -19.718, max: -18.025

Progress of the run: 35 percent (7 of 20 steps)
                           -----norm------ -----index----- -log10(cutoff)-
  Last ensemble median :    2.61e+32          2.66            1.72      
  Last ensemble std    :    1.15e+31          0.119          0.0936     
  Last ensemble lnprob :  avg: -19.554, max: -18.244

Progress of the run: 40 percen

/home/born-again/anaconda3/envs/gammapy-1.1/lib/python3.9/site-packages/naima/analysis.py:163: UserWarning: Attempt to set non-positive xlim on a log-scaled axis will be ignored.
  f.clf()
